# Modelleren van Haaiengroei - Von Bertalanffy Model

Dit notebook bevat alle Python code voor het analyseren en visualiseren van het von Bertalanffy groeimodel voor haaien.

**Auteurs:** Olaf Smits, Konstantinos Pantelakis, Teun van den Berg  
**Instelling:** TU Delft

## Cel 1: Imports en Setup

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from scipy.integrate import odeint

# Plot styling
plt.rcParams['figure.figsize'] = [10, 6]
plt.rcParams['font.size'] = 12
plt.rcParams['axes.grid'] = True

print("Libraries loaded successfully!")

## Cel 2: Analytische Oplossing van de Differentiaalvergelijking

De von Bertalanffy vergelijking:
$$\frac{dw}{dt} = \eta w^{2/3} - \kappa w$$

heeft de analytische oplossing:
$$w(t) = \left[ \frac{\eta}{\kappa} + \left(w_0^{1/3} - \frac{\eta}{\kappa}\right) e^{-\kappa t/3} \right]^3$$

In [ ]:
def analytical_solution(t, eta, kappa, w0):
    """
    Analytische oplossing van de von Bertalanffy differentiaalvergelijking:
    dw/dt = eta * w^(2/3) - kappa * w
    
    Parameters:
    -----------
    t : float or array
        Tijd (jaren)
    eta : float
        Anabole coëfficiënt
    kappa : float
        Katabole coëfficiënt
    w0 : float
        Beginmassa (kg)
    
    Returns:
    --------
    w : float or array
        Massa op tijdstip t (kg)
    """
    w_inf_cbrt = eta / kappa  # w_inf^(1/3)
    w0_cbrt = w0**(1/3)
    return (w_inf_cbrt + (w0_cbrt - w_inf_cbrt) * np.exp(-kappa * t / 3))**3

# Verificatie: vergelijk analytische met numerieke oplossing
def dwdt(w, t, eta, kappa):
    """Differentiaalvergelijking voor numerieke integratie"""
    if w <= 0:
        return 0
    return eta * w**(2/3) - kappa * w

# Test parameters
eta_test = 3.78
kappa_test = 0.30
w0_test = 25.0
t_test = np.linspace(0, 80, 500)

# Analytische oplossing
w_analytical = analytical_solution(t_test, eta_test, kappa_test, w0_test)

# Numerieke oplossing (Euler en scipy odeint)
w_numerical = odeint(dwdt, w0_test, t_test, args=(eta_test, kappa_test)).flatten()

# Evenwichtsoplossing
w_star = (eta_test / kappa_test)**3

print(f"Parameters: eta = {eta_test}, kappa = {kappa_test}")
print(f"Evenwichtsmassa w* = (eta/kappa)^3 = {w_star:.2f} kg")
print(f"Maximale afwijking analytisch vs numeriek: {np.max(np.abs(w_analytical - w_numerical)):.6f} kg")

In [ ]:
# Plot vergelijking analytisch vs numeriek
fig, ax = plt.subplots(figsize=(12, 6))

ax.plot(t_test, w_analytical, 'b-', linewidth=2, label='Analytische oplossing')
ax.plot(t_test, w_numerical, 'r--', linewidth=2, label='Numerieke oplossing (odeint)', alpha=0.7)
ax.axhline(y=w_star, color='green', linestyle=':', linewidth=2, label=f'Evenwicht w* = {w_star:.0f} kg')

ax.set_xlabel('Leeftijd (jaren)', fontsize=14)
ax.set_ylabel('Massa (kg)', fontsize=14)
ax.set_title('Verificatie: Analytische vs Numerieke Oplossing', fontsize=16)
ax.legend(fontsize=12)
ax.set_xlim(0, 80)
ax.set_ylim(0, w_star * 1.1)

plt.tight_layout()
plt.savefig('analytical_vs_numerical.png', dpi=150, bbox_inches='tight')
plt.show()

## Cel 3: Richtingsveld (Direction Field)

Een richtingsveld visualiseert de helling $\frac{dw}{dt}$ voor verschillende combinaties van $t$ en $w$.

In [ ]:
def plot_direction_field(eta, kappa, w0, t_max=80, w_max=3000, n_arrows=20):
    """
    Plot het richtingsveld voor de von Bertalanffy vergelijking.
    
    Parameters:
    -----------
    eta, kappa : float
        Model parameters
    w0 : float
        Beginmassa voor de oplossingscurve
    t_max : float
        Maximum tijd voor de plot
    w_max : float
        Maximum massa voor de plot
    n_arrows : int
        Aantal pijlen per dimensie
    """
    w_star = (eta / kappa)**3
    
    # Grid voor pijlen
    t_grid = np.linspace(0.1, t_max, n_arrows)
    w_grid = np.linspace(10, w_max, n_arrows)
    T, W = np.meshgrid(t_grid, w_grid)
    
    # Berekenen van hellingen
    dWdt = eta * W**(2/3) - kappa * W
    
    # Normaliseren voor visualisatie
    dt = np.ones_like(dWdt)
    magnitude = np.sqrt(dt**2 + (dWdt * t_max/w_max)**2)
    dt_norm = dt / magnitude
    dw_norm = (dWdt * t_max/w_max) / magnitude
    
    # Plot
    fig, ax = plt.subplots(figsize=(12, 8))
    
    # Richtingsveld
    ax.quiver(T, W, dt_norm, dw_norm, color='gray', alpha=0.6, 
              scale=30, headwidth=4, headlength=5, width=0.003)
    
    # Oplossingscurve
    t_sol = np.linspace(0, t_max, 500)
    w_sol = analytical_solution(t_sol, eta, kappa, w0)
    ax.plot(t_sol, w_sol, 'b-', linewidth=3, label=f'Oplossing (w₀ = {w0} kg)')
    
    # Extra oplossingscurves met verschillende beginwaarden
    for w0_extra in [5, 100, w_star * 1.2]:
        w_sol_extra = analytical_solution(t_sol, eta, kappa, w0_extra)
        ax.plot(t_sol, w_sol_extra, '--', linewidth=2, alpha=0.7, 
                label=f'w₀ = {w0_extra:.0f} kg')
    
    # Evenwichtslijn
    ax.axhline(y=w_star, color='red', linestyle=':', linewidth=2, 
               label=f'Evenwicht w* = {w_star:.0f} kg')
    
    ax.set_xlabel('Leeftijd (jaren)', fontsize=14)
    ax.set_ylabel('Massa (kg)', fontsize=14)
    ax.set_title('Richtingsveld: Von Bertalanffy Groeimodel', fontsize=16)
    ax.legend(loc='lower right', fontsize=11)
    ax.set_xlim(0, t_max)
    ax.set_ylim(0, w_max)
    
    plt.tight_layout()
    return fig, ax

# Parameters (gebaseerd op Great White Shark)
eta = 3.78
kappa = 0.30
w0 = 25

fig, ax = plot_direction_field(eta, kappa, w0)
plt.savefig('direction_field.png', dpi=150, bbox_inches='tight')
plt.show()

## Cel 4: Faselijn (Phase Line)

De faselijn toont de dynamiek van het systeem in één dimensie. We plotten $\frac{dw}{dt}$ als functie van $w$.

In [ ]:
def plot_phase_line(eta, kappa, w_max=3000):
    """
    Plot de faselijn en fase-diagram voor de von Bertalanffy vergelijking.
    
    Parameters:
    -----------
    eta, kappa : float
        Model parameters
    w_max : float
        Maximum massa voor de plot
    """
    w_star = (eta / kappa)**3
    
    # Massa range
    w = np.linspace(0.1, w_max, 1000)
    
    # f(w) = dw/dt
    f_w = eta * w**(2/3) - kappa * w
    
    # Create figure with two subplots
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10), 
                                    gridspec_kw={'height_ratios': [3, 1]})
    
    # --- Top plot: f(w) vs w ---
    ax1.plot(w, f_w, 'b-', linewidth=2.5)
    ax1.axhline(y=0, color='black', linewidth=1)
    ax1.axvline(x=w_star, color='red', linestyle='--', linewidth=2, 
                label=f'w* = {w_star:.0f} kg')
    
    # Markeer evenwichtspunten
    ax1.plot(0, 0, 'ko', markersize=12, markerfacecolor='white', markeredgewidth=2)
    ax1.plot(w_star, 0, 'ro', markersize=12)
    
    # Annotaties
    ax1.annotate('Instabiel\nevenwicht', xy=(0, 0), xytext=(100, 30),
                fontsize=11, ha='center',
                arrowprops=dict(arrowstyle='->', color='gray'))
    ax1.annotate('Stabiel\nevenwicht', xy=(w_star, 0), xytext=(w_star - 300, -20),
                fontsize=11, ha='center',
                arrowprops=dict(arrowstyle='->', color='gray'))
    
    # Kleur de regio's
    ax1.fill_between(w, f_w, 0, where=(f_w > 0), alpha=0.3, color='green', 
                     label='Groei (dw/dt > 0)')
    ax1.fill_between(w, f_w, 0, where=(f_w < 0), alpha=0.3, color='red', 
                     label='Afname (dw/dt < 0)')
    
    ax1.set_xlabel('Massa w (kg)', fontsize=14)
    ax1.set_ylabel('Groeisnelheid dw/dt (kg/jaar)', fontsize=14)
    ax1.set_title('Fase-diagram: f(w) = ηw^(2/3) - κw', fontsize=16)
    ax1.legend(loc='upper right', fontsize=11)
    ax1.set_xlim(0, w_max)
    
    # --- Bottom plot: Phase line ---
    ax2.axhline(y=0.5, color='black', linewidth=2)
    
    # Evenwichtspunten
    ax2.plot(0, 0.5, 'ko', markersize=15, markerfacecolor='white', markeredgewidth=3)
    ax2.plot(w_star, 0.5, 'ro', markersize=15)
    
    # Pijlen voor de richting van de flow
    arrow_positions = [w_star/4, w_star/2, w_star*0.75, w_star*1.1]
    for pos in arrow_positions:
        if pos < w_star:
            ax2.annotate('', xy=(pos + 100, 0.5), xytext=(pos, 0.5),
                        arrowprops=dict(arrowstyle='->', color='green', lw=2))
        else:
            ax2.annotate('', xy=(pos - 100, 0.5), xytext=(pos, 0.5),
                        arrowprops=dict(arrowstyle='->', color='red', lw=2))
    
    # Labels
    ax2.text(0, 0.2, 'w = 0\n(instabiel)', ha='center', fontsize=11)
    ax2.text(w_star, 0.2, f'w* = {w_star:.0f} kg\n(stabiel)', ha='center', fontsize=11)
    
    ax2.set_xlabel('Massa w (kg)', fontsize=14)
    ax2.set_title('Faselijn', fontsize=14)
    ax2.set_xlim(0, w_max)
    ax2.set_ylim(0, 1)
    ax2.set_yticks([])
    ax2.grid(False)
    
    plt.tight_layout()
    return fig, (ax1, ax2)

# Plot
fig, axes = plot_phase_line(eta, kappa)
plt.savefig('phase_line.png', dpi=150, bbox_inches='tight')
plt.show()

## Cel 5: Casestudy - Shortfin Mako Haai

We gebruiken gepubliceerde data voor de Shortfin Mako haai (*Isurus oxyrinchus*) om de parameters van het von Bertalanffy model te schatten.

In [ ]:
# Shortfin Mako shark data (female)
# Data based on Rolim et al. (2020) - South Atlantic population
# Length converted to weight using: W = 0.0000044 * L^3.14 (L in cm, W in kg)

# Age-length data from literature
ages = np.array([0, 1, 2, 3, 4, 5, 7, 10, 15, 20, 25])
lengths_cm = np.array([70, 95, 118, 139, 158, 175, 205, 242, 285, 310, 325])

# Convert length to mass
# Using length-weight relationship: W = a * L^b
# For shortfin mako: a ≈ 4.4e-6, b ≈ 3.14
a_lw = 4.4e-6
b_lw = 3.14
masses_kg = a_lw * lengths_cm**b_lw

print("Shortfin Mako Haai - Leeftijd-Massa Data")
print("="*50)
print(f"{'Leeftijd (jaar)':<18} {'Lengte (cm)':<15} {'Massa (kg)':<12}")
print("-"*50)
for age, length, mass in zip(ages, lengths_cm, masses_kg):
    print(f"{age:<18} {length:<15} {mass:<12.1f}")

## Cel 6: Parameter Fitting

We schatten de parameters $\eta$ en $\kappa$ door de kleinste kwadraten methode.

In [ ]:
def fit_vonbertalanffy(t_data, w_data, w0):
    """
    Fit de von Bertalanffy parameters aan data.
    
    Parameters:
    -----------
    t_data : array
        Leeftijden (jaren)
    w_data : array
        Massa's (kg)
    w0 : float
        Beginmassa (kg)
    
    Returns:
    --------
    eta, kappa : float
        Geschatte parameters
    result : OptimizeResult
        Volledige optimalisatieresultaat
    """
    def objective(params):
        eta, kappa = params
        if eta <= 0 or kappa <= 0:
            return 1e10
        w_pred = analytical_solution(t_data, eta, kappa, w0)
        return np.sum((w_pred - w_data)**2)
    
    # Initiële schattingen
    x0 = [2.0, 0.3]
    
    # Optimalisatie met bounds
    result = minimize(objective, x0, method='L-BFGS-B',
                      bounds=[(0.1, 10), (0.01, 1)])
    
    return result.x[0], result.x[1], result

# Fit de parameters
w0_mako = masses_kg[0]  # Geboortemassa
eta_fit, kappa_fit, opt_result = fit_vonbertalanffy(ages, masses_kg, w0_mako)

# Bereken afgeleide grootheden
w_star_fit = (eta_fit / kappa_fit)**3
K_fit = kappa_fit / 3  # Von Bertalanffy groeisnelheid

# R-squared berekenen
w_pred = analytical_solution(ages, eta_fit, kappa_fit, w0_mako)
ss_res = np.sum((masses_kg - w_pred)**2)
ss_tot = np.sum((masses_kg - np.mean(masses_kg))**2)
r_squared = 1 - (ss_res / ss_tot)

print("\nParameterschattingen Shortfin Mako Haai")
print("="*50)
print(f"Anabole coëfficiënt (η):        {eta_fit:.4f}")
print(f"Katabole coëfficiënt (κ):       {kappa_fit:.4f}")
print(f"\nAfgeleide grootheden:")
print(f"Maximale massa (w*):            {w_star_fit:.1f} kg")
print(f"Von Bertalanffy K:              {K_fit:.4f} /jaar")
print(f"\nModel fit:")
print(f"R² (coefficient of determination): {r_squared:.4f}")
print(f"Sum of Squared Errors:          {opt_result.fun:.2f}")

## Cel 7: Visualisatie van de Fit

In [ ]:
def plot_mako_fit(ages, masses, eta, kappa, w0):
    """
    Plot de data en het gefitte model.
    """
    w_star = (eta / kappa)**3
    
    # Tijdsgrid voor model
    t_model = np.linspace(0, 35, 500)
    w_model = analytical_solution(t_model, eta, kappa, w0)
    
    # Figure
    fig, ax = plt.subplots(figsize=(12, 8))
    
    # Data punten
    ax.scatter(ages, masses, s=150, c='blue', edgecolors='black', 
               linewidth=2, zorder=5, label='Waargenomen data')
    
    # Model curve
    ax.plot(t_model, w_model, 'b-', linewidth=2.5, 
            label=f'Von Bertalanffy fit (η={eta:.2f}, κ={kappa:.2f})')
    
    # Asymptote
    ax.axhline(y=w_star, color='red', linestyle='--', linewidth=2,
               label=f'Maximale massa w* = {w_star:.0f} kg')
    
    # Residuals als error bars (optioneel)
    w_pred = analytical_solution(ages, eta, kappa, w0)
    ax.vlines(ages, masses, w_pred, colors='gray', alpha=0.5, linewidth=1)
    
    ax.set_xlabel('Leeftijd (jaren)', fontsize=14)
    ax.set_ylabel('Massa (kg)', fontsize=14)
    ax.set_title('Shortfin Mako Haai (Isurus oxyrinchus) - Model Fit', fontsize=16)
    ax.legend(loc='lower right', fontsize=12)
    ax.set_xlim(-1, 35)
    ax.set_ylim(0, w_star * 1.2)
    
    # Annotatie met R²
    ax.text(0.05, 0.95, f'R² = {r_squared:.4f}', transform=ax.transAxes,
            fontsize=14, verticalalignment='top', 
            bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
    
    plt.tight_layout()
    return fig, ax

fig, ax = plot_mako_fit(ages, masses_kg, eta_fit, kappa_fit, w0_mako)
plt.savefig('mako_fit.png', dpi=150, bbox_inches='tight')
plt.show()

## Cel 8: Residual Analysis

In [ ]:
# Bereken residuals
w_predicted = analytical_solution(ages, eta_fit, kappa_fit, w0_mako)
residuals = masses_kg - w_predicted

# Plot residuals
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Residuals vs Age
axes[0].scatter(ages, residuals, s=100, c='blue', edgecolors='black')
axes[0].axhline(y=0, color='red', linestyle='--')
axes[0].set_xlabel('Leeftijd (jaren)', fontsize=12)
axes[0].set_ylabel('Residual (kg)', fontsize=12)
axes[0].set_title('Residuals vs Leeftijd', fontsize=14)

# Residuals vs Predicted
axes[1].scatter(w_predicted, residuals, s=100, c='blue', edgecolors='black')
axes[1].axhline(y=0, color='red', linestyle='--')
axes[1].set_xlabel('Voorspelde massa (kg)', fontsize=12)
axes[1].set_ylabel('Residual (kg)', fontsize=12)
axes[1].set_title('Residuals vs Voorspelde Waarden', fontsize=14)

plt.tight_layout()
plt.savefig('residuals.png', dpi=150, bbox_inches='tight')
plt.show()

print("\nResidual Statistieken:")
print(f"Gemiddelde residual: {np.mean(residuals):.2f} kg")
print(f"Standaarddeviatie:   {np.std(residuals):.2f} kg")
print(f"Max absolute error:  {np.max(np.abs(residuals)):.2f} kg")

## Cel 9: Parameter Sensitivity Analysis

In [ ]:
# Parameter sensitivity
def sensitivity_analysis(eta_base, kappa_base, w0, t_max=35):
    """
    Analyseer de gevoeligheid van het model voor parameter variaties.
    """
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))
    
    t = np.linspace(0, t_max, 500)
    
    # Variatie in eta
    ax1 = axes[0]
    eta_variations = [0.7, 0.85, 1.0, 1.15, 1.3]
    colors = plt.cm.Blues(np.linspace(0.3, 1, len(eta_variations)))
    
    for i, factor in enumerate(eta_variations):
        eta = eta_base * factor
        w = analytical_solution(t, eta, kappa_base, w0)
        w_inf = (eta / kappa_base)**3
        ax1.plot(t, w, color=colors[i], linewidth=2,
                label=f'η = {factor:.0%} ({w_inf:.0f} kg)')
    
    ax1.set_xlabel('Leeftijd (jaren)', fontsize=12)
    ax1.set_ylabel('Massa (kg)', fontsize=12)
    ax1.set_title(f'Gevoeligheid voor η (κ = {kappa_base:.2f})', fontsize=14)
    ax1.legend(title='η factor (w*)', fontsize=10)
    
    # Variatie in kappa
    ax2 = axes[1]
    kappa_variations = [0.7, 0.85, 1.0, 1.15, 1.3]
    colors = plt.cm.Reds(np.linspace(0.3, 1, len(kappa_variations)))
    
    for i, factor in enumerate(kappa_variations):
        kappa = kappa_base * factor
        w = analytical_solution(t, eta_base, kappa, w0)
        w_inf = (eta_base / kappa)**3
        ax2.plot(t, w, color=colors[i], linewidth=2,
                label=f'κ = {factor:.0%} ({w_inf:.0f} kg)')
    
    ax2.set_xlabel('Leeftijd (jaren)', fontsize=12)
    ax2.set_ylabel('Massa (kg)', fontsize=12)
    ax2.set_title(f'Gevoeligheid voor κ (η = {eta_base:.2f})', fontsize=14)
    ax2.legend(title='κ factor (w*)', fontsize=10)
    
    plt.tight_layout()
    return fig, axes

fig, axes = sensitivity_analysis(eta_fit, kappa_fit, w0_mako)
plt.savefig('sensitivity.png', dpi=150, bbox_inches='tight')
plt.show()

## Cel 10: Temperatuur- en klimaatscenario's

We koppelen het von Bertalanffy model aan temperatuurafhankelijke $Q_{10}$-schaalfactoren om te onderzoeken hoe opwarming de groei van de Shortfin Mako beïnvloedt. Het scenario gebruikt constante temperatuurverschillen (+0, +2 en +4 $^\circ$C) om het effect van opwarming te isoleren.

In [ ]:
# Temperatuur-afhankelijke groeisimulatie
def q10_scale(T, T_ref, Q10):
    return Q10 ** ((T - T_ref) / 10)

MIN_WEIGHT = 0.0

def temperature_growth(eta_ref, kappa_ref, w0, T_series, t, Q10_a=2.0, Q10_k=2.5):
    w = np.zeros_like(t)
    w[0] = w0
    for i in range(1, len(t)):
        eta_t = eta_ref * q10_scale(T_series[i], T_series[0], Q10_a)
        kappa_t = kappa_ref * q10_scale(T_series[i], T_series[0], Q10_k)
        dt = t[i] - t[i-1]
        dwdt = eta_t * w[i-1]**(2/3) - kappa_t * w[i-1]
        w[i] = max(w[i-1] + dt * dwdt, MIN_WEIGHT)
    return w

def asymptotic_mass(eta_ref, kappa_ref, T, T_ref=18, Q10_a=2.0, Q10_k=2.5):
    eta_T = eta_ref * q10_scale(T, T_ref, Q10_a)
    kappa_T = kappa_ref * q10_scale(T, T_ref, Q10_k)
    return (eta_T / kappa_T)**3

T_ref = 18  # graad Celsius
years = np.linspace(0, 40, 400)

Y_AXIS_PADDING = 1.2

# Scenario's met constante temperatuurverschillen
scenarios = [
    ("Huidig (18°C)", np.full_like(years, T_ref)),
    ("+2°C scenario", np.full_like(years, T_ref + 2)),
    ("+4°C scenario", np.full_like(years, T_ref + 4)),
]

results = []
for label, temps in scenarios:
    w_path = temperature_growth(eta_fit, kappa_fit, w0_mako, temps, years)
    w_inf = asymptotic_mass(eta_fit, kappa_fit, temps[-1], T_ref=T_ref)
    results.append((label, temps, w_path, w_inf))

fig, ax = plt.subplots(figsize=(12, 7))
colors = ['#1f77b4', '#ff7f0e', '#2ca02c']

for color, (label, temps, w_path, w_inf) in zip(colors, results):
    ax.plot(years, w_path, color=color, linewidth=2.5,
            label=f"{label} (w* ≈ {w_inf:.0f} kg)")
    ax.axhline(w_inf, color=color, linestyle='--', alpha=0.4)

ax.scatter(ages, masses_kg, s=80, c='black', alpha=0.6, zorder=5, label='Data')
ax.set_xlabel('Leeftijd (jaren)', fontsize=13)
ax.set_ylabel('Massa (kg)', fontsize=13)
ax.set_title('Temperatuur- en klimaatscenario\'s voor Shortfin Mako', fontsize=15)
ax.legend(loc='lower right')
ax.set_xlim(0, years.max())
ax.set_ylim(0, max(w_inf for *_, w_inf in results) * Y_AXIS_PADDING)
plt.tight_layout()
plt.savefig('temperature_scenarios.png', dpi=150, bbox_inches='tight')
plt.show()

print("Evenwichtsmassa per scenario (Q10_a=2.0, Q10_k=2.5):")
for label, temps, _, w_inf in results:
    print(f"- {label}: w* ≈ {w_inf:.1f} kg")

## Cel 11: Vergelijking met Euler's Methode

Vergelijk de analytische oplossing met numerieke integratie via Euler's methode.

In [ ]:
def euler_method(eta, kappa, w0, t_end, dt):
    """
    Los de differentiaalvergelijking op met Euler's methode.
    
    Parameters:
    -----------
    eta, kappa : float
        Model parameters
    w0 : float
        Beginmassa
    t_end : float
        Eindtijd
    dt : float
        Tijdstap
    
    Returns:
    --------
    t_arr, w_arr : arrays
        Tijd en massa arrays
    """
    n_steps = int(t_end / dt)
    t_arr = np.zeros(n_steps + 1)
    w_arr = np.zeros(n_steps + 1)
    
    t_arr[0] = 0
    w_arr[0] = w0
    
    for i in range(1, n_steps + 1):
        w = w_arr[i-1]
        dwdt = eta * w**(2/3) - kappa * w
        w_arr[i] = w + dt * dwdt
        t_arr[i] = t_arr[i-1] + dt
    
    return t_arr, w_arr

# Vergelijk verschillende stapsizes
dt_values = [0.5, 0.1, 0.01]
t_end = 35

fig, ax = plt.subplots(figsize=(12, 7))

# Analytische oplossing
t_exact = np.linspace(0, t_end, 1000)
w_exact = analytical_solution(t_exact, eta_fit, kappa_fit, w0_mako)
ax.plot(t_exact, w_exact, 'k-', linewidth=3, label='Analytisch')

# Euler met verschillende dt
colors = ['blue', 'green', 'red']
for dt, color in zip(dt_values, colors):
    t_euler, w_euler = euler_method(eta_fit, kappa_fit, w0_mako, t_end, dt)
    ax.plot(t_euler, w_euler, '--', color=color, linewidth=2, 
            label=f'Euler (dt = {dt})')

ax.scatter(ages, masses_kg, s=100, c='orange', edgecolors='black', 
           zorder=5, label='Data')

ax.set_xlabel('Leeftijd (jaren)', fontsize=14)
ax.set_ylabel('Massa (kg)', fontsize=14)
ax.set_title('Vergelijking: Analytisch vs Euler Methode', fontsize=16)
ax.legend(fontsize=11)

plt.tight_layout()
plt.savefig('euler_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

## Cel 11: Samenvatting en Conclusies

In [ ]:
print("="*60)
print("SAMENVATTING: Von Bertalanffy Model voor Shortfin Mako Haai")
print("="*60)

print("\n1. MODEL")
print("-"*60)
print("Differentiaalvergelijking: dw/dt = η·w^(2/3) - κ·w")
print("Analytische oplossing: w(t) = [η/κ + (w₀^(1/3) - η/κ)·e^(-κt/3)]³")

print("\n2. GESCHATTE PARAMETERS")
print("-"*60)
print(f"Anabole coëfficiënt η = {eta_fit:.4f}")
print(f"Katabole coëfficiënt κ = {kappa_fit:.4f}")
print(f"Maximale massa w* = {w_star_fit:.1f} kg")

print("\n3. MODEL FIT")
print("-"*60)
print(f"R² = {r_squared:.4f}")
print(f"Root Mean Square Error = {np.sqrt(np.mean(residuals**2)):.2f} kg")

print("\n4. BIOLOGISCHE INTERPRETATIE")
print("-"*60)
print("- De Shortfin Mako bereikt asymptotisch ~417 kg")
print("- De groeisnelheid K = κ/3 ≈ 0.10 /jaar")
print("- Het model past goed bij de waargenomen groeipatronen")

print("\n5. STABILITEITSANALYSE")
print("-"*60)
print("- Evenwicht w = 0 is instabiel")
print(f"- Evenwicht w* = {w_star_fit:.0f} kg is asymptotisch stabiel")
print("- f'(w*) = -κ/3 < 0, dus alle oplossingen convergeren naar w*")

---

## Bijlage: Alle Figuren Opslaan

In [ ]:
# Run dit cell om alle figuren opnieuw te genereren en op te slaan

# 1. Direction field
fig, ax = plot_direction_field(eta_fit, kappa_fit, w0_mako, t_max=35, w_max=500)
plt.savefig('direction_field.png', dpi=150, bbox_inches='tight')
plt.close()

# 2. Phase line
fig, axes = plot_phase_line(eta_fit, kappa_fit, w_max=600)
plt.savefig('phase_line.png', dpi=150, bbox_inches='tight')
plt.close()

# 3. Mako fit
fig, ax = plot_mako_fit(ages, masses_kg, eta_fit, kappa_fit, w0_mako)
plt.savefig('mako_fit.png', dpi=150, bbox_inches='tight')
plt.close()

print("Alle figuren zijn opgeslagen:")
print("- direction_field.png")
print("- phase_line.png")
print("- mako_fit.png")